In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os, sys
sys.path.append('/content/drive/MyDrive/CPD_BT')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import random

import itertools

from bt_cpd import *

import time
import bisect

import pandas as pd

import statsmodels.api as sm
from sklearn import linear_model

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
T = 4
Delta = 500
m = np.array([Delta] * T)
cp_truth = np.cumsum(m)[:T-1]

n = 10
beta = np.zeros((T, n))

t = 0.9
kappa = np.log(t / (1 - t))
delta = 1
beta_ref = get_beta_with_gap(n, delta)
beta_ref *= kappa / (np.max(beta_ref) - np.min(beta_ref))
beta[0] = beta_ref[:]
beta[1] = change_type(beta_ref, 1)
beta[2] = change_type(beta_ref, 2)
beta[3] = change_type(beta_ref, 3)

print(max(beta[0]) - min(beta[0]))

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(beta[t] - beta[t - 1])**2)**0.5
print(diff)

2.1972245773362196
[4.43495185 3.86013011 4.43495185]


In [5]:
cp_truth

array([ 500, 1000, 1500])

In [6]:
path = '/content/drive/MyDrive/CPD_BT/'
with open(path + 'data_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '.npy', 'rb') as f:
    X_train_list = np.load(f)
    Y_train_list = np.load(f)
    X_test_list = np.load(f)
    Y_test_list = np.load(f)

In [7]:
X_train_list.shape

(100, 2000, 10)

In [8]:
np.random.seed(0)

grid_n = 90
gamma_list = [20, 40]
lam_list = [0.1]

nt = Delta * T
B = 100

run_time_d = np.zeros(B)
loc_error_d = np.zeros(B)
K_d = np.zeros(B)

cp_best_list = []
param_best_list = []
cp_best_cand_list = []

for b in range(B):
    X_train = X_train_list[b]
    Y_train = Y_train_list[b]
    X_test = X_test_list[b]
    Y_test = Y_test_list[b]

    start_time = time.time()
    dp_fit = dplr_cv_bt(grid_n, lam_list, gamma_list)
    cp_best, param_best, cp_best_cand = dp_fit.fit((Y_train, X_train), (Y_test, X_test))    
    run_time_d[b] = time.time() - start_time
    loc_error_d[b] = cp_distance(cp_best, cp_truth)
    K_d[b] = len(cp_best)

    cp_best_list.append(cp_best)
    param_best_list.append(param_best)
    cp_best_cand_list.append(cp_best_cand)
    print(b)


print('---------- dplr -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_d.mean(), run_time_d.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_d.std(), run_time_d.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_d < T - 1), sum(K_d == T - 1), sum(K_d > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- dplr -----------
avg loc error: 9.24, avg time: 49.65604029417038
std loc error: 9.141247179679587, std time: 0.6756706020789961
K < K*: 0, K = K*: 100, K > K*: 0


In [9]:
loc_error_d

array([ 9.,  2., 16.,  7.,  2.,  1.,  8.,  7.,  8., 25.,  9., 16., 48.,
        5.,  3.,  4.,  2., 12.,  3.,  2.,  1.,  7., 31.,  3.,  0.,  2.,
       35.,  3.,  9.,  8., 29., 39.,  8.,  5.,  7.,  6., 22.,  1.,  1.,
        1.,  8.,  5., 11.,  3.,  3.,  2.,  9., 14.,  2.,  7., 20., 24.,
        9.,  4.,  4.,  6.,  1.,  6.,  5.,  5., 27.,  2., 22., 13., 13.,
        4.,  4.,  3.,  5., 14.,  2., 16., 19.,  9.,  3., 13.,  7.,  1.,
        1., 10., 10., 11., 10.,  1.,  8., 10.,  2.,  8.,  3.,  3.,  2.,
        4., 22.,  4., 19., 31.,  6.,  1., 11., 15.])

In [10]:
cp_best_list[0]

[492, 991, 1500]

In [11]:
cp_best_cand_list[0]

array([ 484,  990, 1496])

In [12]:
import pickle
with open(path + 'dplr_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_grid' + str(grid_n) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, param_best_list, cp_best_cand_list, loc_error_d, run_time_d, K_d], f)